# Modelling spreading of infectious diseases
This is an experimental model. Inspirered by https://triplebyte.com/blog/modeling-infectious-diseases. 

The model is purely for testing the capabilities of ModelFlow, the parameters selected are for ilustration of the dynamic and are not actual estimates. 

This is a Jupyter Notebook running Python. 

The notebook is located on github here: https://github.com/IbHansen/Modelflow_test

Feel free to use this notebook. **To run the notebook** find the menu line above and select **cell>run all** 

THE Notebook IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

The model is specified as equations in the Python code below (you don't have to understand the Python code). The equations defines the daily transition between the states which the population can exist in:

 - susciptible
 - exposed
 - infectious
 - recovered
 - dead 

# Make eksperiments 
The model will run with a baseline scenario and you can inspect the results. 

Then you can use the sliders to change the scenario.
You can make your own scenario. Use the sliders to define an scenario. Then press the **Run scenario** button. The model will solve, and the results can be inspected.

For instance, try to increase *Daily rate of contact* to 6 and press **Run scenario**  and watch how the values changes.

You can change the scenario name. After **Run scenario** a graph for each scenario will be shown. 

Again, you don't have to understand the Python code. 

In [6]:
import pandas as pd

from modelsandbox      import create_new_model
from modeljupyter      import inputwidget
import modelmanipulation as mp

In [29]:
def lag_manual(values,weights,debug=0):
    weights_s = pd.Series(weights)
    weights_normalized = weights_s/weights_s.sum()
    values_s  = pd.Series(values)
    weighted  = (values_s*weights_normalized).fillna(0)
    result = weighted.sum()
    if int(debug):
        weights_org = pd.Series(weights)
        temp = pd.concat([weights_s, weights_normalized,values_s,weighted],axis=1)
        temp.columns = ['weights_org','weights normallized','Values','weighted values']
        time = pd.Series([int(i) for i in temp.index])
        average_lag = (pd.Series(time)*weights_normalized).fillna(0).sum()
        print(temp)
        print(f'Result       {result:15.5f}')
        print(f'Average lag  {average_lag:15.5f}')
        # print(time)
    return result 


In [49]:
 
# Specify the model 
#    Some conventions:
#       - (-1) after a variable means the value the day before.
#       - diff means the change in variable from the day before

rcorona = '''\
             infection_rate        = rate_contact * probability_transmision * infectious(-1) / population(-1)
             new_exposed           = infection_rate * susceptible + exo_exposed
             diff(exposed)         = new_exposed - new_infectious
             
             new_infectious        = new_exposed(-2)*0.8 + new_exposed(-3)*0.2 
             
             new_infectious_to_recover      = new_infectious * new_infectious_to_recover_rate
             exit_infectious_to_recover     = new_infectious_to_recover(-14)
             
             new_infectious_to_hospital     = new_infectious * (1-new_infectious_to_recover_rate)
             diff(infectious)      = new_infectious - exit_infectious_to_recover-new_infectious_to_hospital 
             
             
             new_hospital_to_recover        =  new_infectious_to_hospital * new_hospital_to_recover_rate
             exit_hospital_to_recover       = new_hospital_to_recover(-14)
             diff(hospital_to_recover)      = new_hospital_to_recover - exit_hospital_to_recover
              
             new_hospital_to_Intensive      =  new_infectious_to_hospital * (1-new_hospital_to_recover_rate)
             
             new_Intensive_to_recover       = new_hospital_to_Intensive * new_Intensive_to_recover_rate 
             exit_intensive_to_recover      = new_intensive_to_recover(-20)
             diff(intensive_to_recover)     = new_intensive_to_recover-exit_intensive_to_recover 
             
             new_Intensive_to_dead          = new_hospital_to_Intensive * (1-new_Intensive_to_recover_rate)              
             exit_intensive_to_dead         = new_intensive_to_dead(-20)
             diff(intensive_to_dead)     = new_intensive_to_dead-exit_intensive_to_dead 

             diff(hospital_non_icu)  = new_hospital_to_recover - exit_Hospital_to_recover
             diff(hospital_icu)      = new_Intensive_to_recover+new_Intensive_to_dead-(exit_intensive_to_recover+exit_Intensive_to_dead)
              
             diff(dead) = exit_intensive_to_dead
             
                  
             

             diff(susceptible)     = -new_exposed  

             
             diff(recovered)       = exit_infectious_to_recover + exit_intensive_to_recover + exit_hospital_to_recover
             
             diff(population)      = -diff(dead)             
'''
f0corona = mp.doable(rcorona)
f1corona = mp.explode(f0corona)
fcorona = mp.lagarray_unroll(f1corona,funks=[lag_manual])
mcorona = create_new_model(fcorona)       # create a model instance which can solve the model 

DAYS = 500                                # number of days the model will run 
basedf = pd.DataFrame(index=range(DAYS))  # make an empty dataframe with DAYS rows
basedf.index.name = 'Day'
# mcorona.drawmodel(sink='DEAD',source='EXO_EXPOSED',lag=True)

In [50]:
# Define user interface 
cow = inputwidget(mcorona,basedf,modelopt={'silent':0,'antal':1000,'first_test':10},
                  slidedef = {
                     'Population            '     :{'var' : 'POPULATION SUSCEPTIBLE', 'min' : 0.0, 'max' : 10_000_000, 'value' : 1_000_000,'step':100_000,'op':'=start-','dec':0},
                     'Number of infected t=1'     :{'var' : 'EXO_EXPOSED',            'min' : 0.0, 'max' : 1000,       'value' : 1000,      'step':10,'op':'=impulse','dec':0},
                     'Share of infected with mild symptom' :{'var' : 'NEW_INFECTIOUS_TO_RECOVER_RATE',        'min' : 0.0, 'max' : 1.0,        'value' : 0.1,                'op':'='},
                     'Share in hospital no ICU and recover': {'var' : 'NEW_HOSPITAL_TO_RECOVER_RATE',              'min' : 0.0, 'max' : 1.0,        'value' : 0.01,               'op':'=' },
                     'Share in ICU which recovers'        :{'var' : 'NEW_INTENSIVE_TO_RECOVER_RATE',          'min' : 0.0, 'max' : 1.0,        'value' : 0.1,                'op':'='},
                     'Daily rate of contact'      :{'var' : 'RATE_CONTACT',           'min' : 0.0, 'max' : 30,         'value' : 4,'step':0.1,         'op':'='},
                     'Probability of transmission':{'var' : 'PROBABILITY_TRANSMISION','min' : 0.0, 'max' : 1.0,        'value' : 0.05,'step':0.005,  'op':'=','dec':3},
                             },
                 varpat='infectious recovered dead hospital*',showvar=False)

In [45]:
mcorona['inf* new_inf* exit_inf*'].df.head(20).style.format('{0:.0f}')

,INFECTION_RATE,INFECTIOUS,NEW_INFECTIOUS,NEW_INFECTIOUS_TO_HOSPITAL,NEW_INFECTIOUS_TO_RECOVER,NEW_INFECTIOUS_TO_RECOVER_RATE,EXIT_INFECTIOUS_TO_RECOVER
Day,,,,,,,
20,0,0,0,0,0,0,0
21,0,0,0,0,0,0,0
22,0,80,800,720,80,0,0
23,0,100,200,180,20,0,0
24,0,100,0,0,0,0,0
25,0,101,13,12,1,0,0
26,0,103,19,17,2,0,0
27,0,105,20,18,2,0,0
28,0,107,20,18,2,0,0
